In [1]:
pip install --upgrade --quiet   langchain-community langchain-groq neo4j python-dotenv langchain-neo4j

Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
from dotenv import load_dotenv
load_dotenv()

# Access the credentials
NEO4J_URI = {os.getenv("NEO4J_URI")}
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
print(NEO4J_URI)


neo4j+s://5b9542e8.databases.neo4j.io


In [2]:
import os


os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD
os.environ["GROQ_API_KEY"] = NEO4J_PASSWORD



In [3]:
pip install langchain_neo4j

Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain_community.graphs import Neo4jGraph
print(str(NEO4J_URI))
graph=Neo4jGraph(
    url=str(NEO4J_URI),
    username=str(NEO4J_USERNAME),
    password=str(NEO4J_PASSWORD),
)
graph

neo4j+s://5b9542e8.databases.neo4j.io


Unable to retrieve routing information


ValueError: Could not connect to Neo4j database. Please ensure that the url is correct

In [5]:
%pip install -U langchain-ollama

In [35]:
# from langchain_ollama.llms import OllamaLLM
# llm = OllamaLLM(model="llama3.2")
# run = model.invoke("what is langchain?")

In [33]:
%pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [15]:
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=GROQ_API_KEY,model_name="mixtral-8x7b-32768")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021AB66FAA50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021AB66FB1D0>, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
import requests
import pandas as pd
from langchain.schema import Document
from io import StringIO
from typing import Dict
from dataclasses import dataclass, asdict

class ImmutableDict(Dict):
    def __hash__(self):
        return hash(tuple(sorted(self.items())))

# Monkey patch Document's __repr__ to only show page_content
def custom_repr(self):
    return f"Document(page_content='{self.page_content}')"

# Store the original __repr__ in case we need to restore it
original_repr = Document.__repr__
Document.__repr__ = custom_repr

# List of dataset file URLs
dataset_urls = [
    "https://raw.githubusercontent.com/parth1609/RAGveda/main/dataset/Gita_questions.csv",
    "https://raw.githubusercontent.com/parth1609/RAGveda/main/dataset/PYS_English_Questions.csv"
]

# Initialize an empty list to store documents from all datasets
all_documents = []

for url in dataset_urls:
    try:
        # Fetch the CSV file
        response = requests.get(url)
        response.raise_for_status()

        # Load CSV content into pandas DataFrame
        csv_content = StringIO(response.text)
        df = pd.read_csv(csv_content)

        # Convert DataFrame rows to Document objects
        dataset_documents = []
        for index, row in df.iterrows():
            text = " | ".join([f"{col}: {row[col]}" for col in df.columns])
            # Create Document with hashable metadata dictionary
            metadata = ImmutableDict({"source_index": index})
            document = Document(
                page_content=text,
                metadata=metadata
            )
            dataset_documents.append(document)
        
        all_documents.extend(dataset_documents)
        print(f"Processed {len(dataset_documents)} documents from {url.split('/')[-1]}")
        
        # Show example of document representation
        if dataset_documents:
            print("\nExample document representation:")
            print(dataset_documents[0])

    except Exception as e:
        print(f"Error loading {url}: {e}")

print(f"\nTotal documents processed: {len(all_documents)}")

Processed 700 documents from Gita_questions.csv

Example document representation:
page_content='chapter: 1 | verse: 1 | speaker: धृतराष्ट्र | sanskrit: धर्मक्षेत्रे कुरुक्षेत्रे समवेता युयुत्सवः| मामकाः पाण्डवाश्चैव किमकुर्वत सञ्जय  || 1.1 ||  | translation: Dhritarashtra said, "What did my people and the sons of Pandu do when they had assembled together, eager for battle, on the holy plain of Kurukshetra, O Sanjaya?" | question: How does the Gita start?' metadata={'source_index': 0}
Processed 195 documents from PYS_English_Questions.csv

Example document representation:
page_content='chapter: 1 | verse: 1 | sanskrit: अथ योगानुशासनम् | translation: Now, the teachings of yoga are presented since the student is ready to received these teachings. | question: When does Yoga commence? When should I start doing Yoga?' metadata={'source_index': 0}

Total documents processed: 895


In [37]:
a = [all_documents[0]]
a

[Document(page_content='chapter: 1 | verse: 1 | speaker: धृतराष्ट्र | sanskrit: धर्मक्षेत्रे कुरुक्षेत्रे समवेता युयुत्सवः| मामकाः पाण्डवाश्चैव किमकुर्वत सञ्जय  || 1.1 ||  | translation: Dhritarashtra said, "What did my people and the sons of Pandu do when they had assembled together, eager for battle, on the holy plain of Kurukshetra, O Sanjaya?" | question: How does the Gita start?')]

In [9]:
!pip install --upgrade --quiet langchain_experimental 

In [10]:
pip install json-repair

Note: you may need to restart the kernel to use updated packages.


In [ ]:


from langchain_experimental.graph_transformers import LLMGraphTransformer

try:
    llm_transformer = LLMGraphTransformer(llm=llm)
    print("Transformer created successfully.")
except Exception as e:
    print(f"Error: {e}")


ImportError: cannot import name 'add_graph_documents' from 'langchain_experimental.graph_transformers' (c:\Users\parth\OneDrive\Desktop\one\RAGveda\my_env\Lib\site-packages\langchain_experimental\graph_transformers\__init__.py)

In [69]:
try:
    graph_doc = llm_transformer.convert_to_graph_documents(a)
    print("Graph documents generated successfully.")
except Exception as e:
    print(f"Error generating graph documents: {e}")

Graph documents generated successfully.


In [70]:
graph_doc

[GraphDocument(nodes=[Node(id='Sanjaya', type='Person', properties={}), Node(id='Dhritarashtra', type='Person', properties={})], relationships=[Relationship(source=Node(id='Dhritarashtra', type='Person', properties={}), target=Node(id='Sanjaya', type='Person', properties={}), type='ASKED', properties={})], source=Document(page_content='chapter: 1 | verse: 1 | speaker: धृतराष्ट्र | sanskrit: धर्मक्षेत्रे कुरुक्षेत्रे समवेता युयुत्सवः| मामकाः पाण्डवाश्चैव किमकुर्वत सञ्जय  || 1.1 ||  | translation: Dhritarashtra said, "What did my people and the sons of Pandu do when they had assembled together, eager for battle, on the holy plain of Kurukshetra, O Sanjaya?" | question: How does the Gita start?'))]

In [71]:
graph_doc[0].relationships

[Relationship(source=Node(id='Dhritarashtra', type='Person', properties={}), target=Node(id='Sanjaya', type='Person', properties={}), type='ASKED', properties={})]

In [3]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True)
chain

ModuleNotFoundError: Module langchain_community.chains.graph_qa.cypher not found. Please install langchain-community to access this module. You can install it using `pip install -U langchain-community`

In [5]:

!%pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 3.2 MB/s eta 0:00:00


In [6]:
query = """
// Create Knowledge node
MERGE (knowledge:Knowledge {name: 'Knowledge'})

// Create and connect PYS
MERGE (pys:PYS {name: 'PYS'})
MERGE (knowledge)-[:HAS_SOURCE]->(pys)

// Create and connect GITA
MERGE (gita:GITA {name: 'GITA'})
MERGE (knowledge)-[:HAS_SOURCE]->(gita)
"""


In [7]:
graph.query(query)

[]

In [8]:

query = """
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/parth1609/RAGveda/main/dataset/Gita_questions.csv' AS row
WITH row
WHERE row.chapter IS NOT NULL AND row.chapter <> ''
  AND row.verse IS NOT NULL AND row.verse <> ''

// Ensure the Knowledge node exists
MERGE (knowledge:Knowledge {name: 'Knowledge'})

// Create the GITA label and connect it to Knowledge
MERGE (gita:GITA {name: 'GITA'})
MERGE (knowledge)-[:HAS_SOURCE]->(gita)

// Create and relate Chapter nodes
MERGE (chapter:GITAChapter {number: toInteger(row.chapter)})
MERGE (gita)-[:HAS_CHAPTER]->(chapter)

// Create and relate Verse nodes
MERGE (verse:GITAVerse {number: toInteger(row.verse), chapterNumber: toInteger(row.chapter)})
MERGE (chapter)-[:HAS_VERSE]->(verse)

// Relate Sanskrit text to Verse
MERGE (shloka:GITASanskrit {text: row.sanskrit})
MERGE (verse)-[:HAS_SHLOKA]->(shloka)

// Create Translation node and relate it to Sanskrit Shloka and Verse
MERGE (translation:GITATranslation {text: row.translation})
MERGE (shloka)-[:HAS_TRANSLATION]->(translation)
MERGE (verse)-[:HAS_TRANSLATION]->(translation)

// Create Question node and ensure proper relationships
MERGE (question:GITAQuestion {text: row.question})
MERGE (question)-[:ABOUT_VERSE]->(verse)
MERGE (question)-[:ABOUT_TRANSLATION]->(translation)
MERGE (question)-[:ABOUT_SHLOKA]->(shloka)



"""


In [9]:
graph.query(query)

[]

In [10]:
query="""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/parth1609/RAGveda/main/dataset/PYS_English_Questions.csv' AS row
WITH row
WHERE row.chapter IS NOT NULL AND row.chapter <> ''
  AND row.verse IS NOT NULL AND row.verse <> ''

// Ensure the Knowledge node exists
MERGE (knowledge:Knowledge {name: 'Knowledge'})

// Create the PYS label and connect it to Knowledge
MERGE (pys:PYS {name: 'PYS'})
MERGE (knowledge)-[:HAS_SOURCE]->(pys)

// Create Chapter node and relate it to PYS
MERGE (chapter:PYSChapter {number: toInteger(row.chapter)})
MERGE (pys)-[:HAS_CHAPTER]->(chapter)

// Create Verse node and relate it to Chapter
MERGE (verse:PYSVerse {number: toInteger(row.verse), chapterNumber: toInteger(row.chapter)})
MERGE (chapter)-[:HAS_VERSE]->(verse)

// Relate Sanskrit text to Verse
MERGE (shloka:PYSSanskrit {text: row.sanskrit})
MERGE (verse)-[:HAS_SHLOKA]->(shloka)

// Create Translation node and relate it to Sanskrit Shloka and Verse
MERGE (translation:PYSTranslation {text: row.translation})
MERGE (shloka)-[:HAS_TRANSLATION]->(translation)
MERGE (verse)-[:HAS_TRANSLATION]->(translation)

// Create Question node and ensure proper relationships
MERGE (question:PYSQuestion {text: row.question})
MERGE (question)-[:ABOUT_VERSE]->(verse)
MERGE (question)-[:ABOUT_TRANSLATION]->(translation)
MERGE (question)-[:ABOUT_SHLOKA]->(shloka)


"""

In [11]:
graph.query(query)

[]

In [12]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Knowledge {name: STRING}
PYS {name: STRING}
GITA {name: STRING}
GITAChapter {number: INTEGER}
GITAVerse {number: INTEGER, chapterNumber: INTEGER}
GITASanskrit {text: STRING}
GITATranslation {text: STRING}
GITAQuestion {text: STRING}
PYSChapter {number: INTEGER}
PYSVerse {number: INTEGER, chapterNumber: INTEGER}
PYSSanskrit {text: STRING}
PYSTranslation {text: STRING}
PYSQuestion {text: STRING}
Relationship properties:

The relationships:
(:Knowledge)-[:HAS_SOURCE]->(:PYS)
(:Knowledge)-[:HAS_SOURCE]->(:GITA)
(:PYS)-[:HAS_CHAPTER]->(:PYSChapter)
(:GITA)-[:HAS_CHAPTER]->(:GITAChapter)
(:GITAChapter)-[:HAS_VERSE]->(:GITAVerse)
(:GITAVerse)-[:HAS_SHLOKA]->(:GITASanskrit)
(:GITAVerse)-[:HAS_TRANSLATION]->(:GITATranslation)
(:GITASanskrit)-[:HAS_TRANSLATION]->(:GITATranslation)
(:GITAQuestion)-[:ABOUT_VERSE]->(:GITAVerse)
(:GITAQuestion)-[:ABOUT_TRANSLATION]->(:GITATranslation)
(:GITAQuestion)-[:ABOUT_SHLOKA]->(:GITASanskrit)
(:PYSChapter)-[:HAS_VERSE]->(:PYSVerse)
(:PYSVerse

In [13]:
pip install -U --quiet langchain-nvidia-ai-endpoints

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 39.9 MB/s eta 0:00:00


In [14]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(
  model="meta/llama-3.1-70b-instruct",
  api_key=NVIDIA_API_KEY,

)


In [15]:
%pip install -qU langchain-groq

In [16]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(llm=llm, graph=graph, verbose=True, allow_dangerous_requests=True)



In [20]:
question = "Who was the king from Kashi who fought in the Mahabharata war?"
sanskritShloka = "सर्वस्य चाहं हृदि सन्निविष्टो मत्तः स्मृतिर्ज्ञानमपोहनं च| वेदैश्च सर्वैरहमेव वेद्यो वेदान्तकृद्वेदविदेव चाहम् || 15.15 || "
speaker = "भगवान"

try:
    response = chain.invoke({"query": f"I have this '{question}'. please give about shloka from 'Gita' or 'Pys' particular chapter , along with its structure as[shloka,tranlation], or state that no relevant information exists`"})
    print(response)  # Or display response in a suitable way for your notebook
except Exception as e:
    print(f"Error: {e}") # printing the error that has occurred
    print(f"No relevant information found for the question: {question}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (q:GITAQuestion {text: "Who was the king from Kashi who fought in the Mahabharata war?"})-[:ABOUT_VERSE]->(v:GITAVerse)-[:HAS_SHLOKA]->(s:GITASanskrit)-[:HAS_TRANSLATION]->(t:GITATranslation)
OPTIONAL MATCH (q)-[:ABOUT_SHLOKA]->(s2:GITASanskrit)-[:HAS_TRANSLATION]->(t2:GITATranslation)
RETURN [[s.text, t.text]] as result
UNION
MATCH (q:PYSQuestion {text: "Who was the king from Kashi who fought in the Mahabharata war?"})-[:ABOUT_VERSE]->(v:PYSVerse)-[:HAS_SHLOKA]->(s:PYSSanskrit)-[:HAS_TRANSLATION]->(t:PYSTranslation)
OPTIONAL MATCH (q)-[:ABOUT_SHLOKA]->(s2:PYSSanskrit)-[:HAS_TRANSLATION]->(t2:PYSTranslation)
RETURN [[s.text, t.text]] as result

Full Context:
[{'result': [['धृष्टकेतुश्चेकितानः काशिराजश्च वीर्यवान्| पुरुजित्कुन्तिभोजश्च शैब्यश्च नरपुङ्गवः  || 1.5 || ', 'Dhrishtaketu, Chekitana, the valiant king of Kasi, Purujit, Kuntibhoja, and Saibya—the best of men.']]}]

> Finished chain.
{'query': "I have this 'Who